# Adaptive Cards Toolkit: 05b - Content Templates

This notebook demonstrates how to create and use content templates for articles, rich media, and structured information displays. Please run 01_toolkit_common.ipynb first.

## Setup

First, we need to import the common utilities defined in the toolkit_common notebook as well as all necessary dependencies.

In [ ]:
# Run common utilities notebook (optional, can be skipped if already run in session)
%run 01_toolkit_common.ipynb

# Ensure we have all the necessary imports
import json
import sys
import os
from datetime import datetime

# Add the parent directory to the path so we can import the toolkit
if not os.path.abspath('..') in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

# Import necessary modules directly
from src.adaptive_cards_toolkit.core.element_factory import ElementFactory
from src.adaptive_cards_toolkit.core.layout_helper import LayoutHelper
from src.adaptive_cards_toolkit.templates.templates import TemplateFactory
from src.adaptive_cards_toolkit.core.data_connector import DataConnector
import adaptive_cards.card_types as types
from adaptive_cards.card import AdaptiveCard

# Define our own validation utility since the imported one might have issues
class SimpleValidator:
    def validate(self, card):
        """Simple validation for cards - just calculates size"""
        try:
            card_json = card.to_json()
            size_kb = len(card_json) / 1024.0
            return {
                "valid": True,
                "size": size_kb,
                "details": [],
                "warnings": []
            }
        except Exception as e:
            return {
                "valid": False,
                "size": 0,
                "details": [str(e)],
                "warnings": []
            }

# Create an instance of our simple validator
simple_validator = SimpleValidator()

# Define display_card function to use our simple validator
def display_card(card, title=None):
    """Display a card as JSON with basic validation.
    
    Args:
        card: The card object to display
        title: Optional title to display with the card
    """
    if title:
        print(f"\n## {title}")
    try:
        # Get JSON from the card
        if hasattr(card, 'to_json'):
            card_json = json.loads(card.to_json())
        elif hasattr(card, 'card_data'):
            card_json = card.card_data
        else:
            card_json = card
            
        # Print the JSON
        print(json.dumps(card_json, indent=2))
        
        # Validate card
        if hasattr(card, 'to_json'):
            validation = simple_validator.validate(card)
            print(f"\nCard size: {validation['size']:.2f}KB | Valid: {validation['valid']}")
    except Exception as e:
        print(f"Error displaying card: {e}")

## Introduction to Content Templates

Content templates are designed for presenting information in a structured, readable format. Unlike notifications or confirmations that are focused on actions, content templates prioritize readability and information organization. They're ideal for:

- News articles and blog posts
- Knowledge base entries
- Documentation pages
- Product descriptions
- Team or project introductions

Let's start with a basic article template function.

In [ ]:
def create_article_template(title, content, image_url=None, author=None, date=None, actions=None):
    """Create a basic article template with title, content, and optional image.
    
    Args:
        title (str): Article title
        content (str): Article content (supports basic markdown)
        image_url (str, optional): URL to a header image
        author (str, optional): Article author
        date (str, optional): Publication date
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create card data with JSON construction pattern from style guide
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add header image if provided
        if image_url:
            card_data["body"].append({
                "type": "Image",
                "url": image_url,
                "altText": f"Image for {title}",
                "size": "stretch"
            })
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata if provided
        metadata = []
        if author:
            metadata.append(f"By {author}")
        if date:
            metadata.append(f"Published: {date}")
            
        if metadata:
            metadata_text = " · ".join(metadata)
            card_data["body"].append({
                "type": "TextBlock",
                "text": metadata_text,
                "isSubtle": True,
                "wrap": True
            })
            
        # Add content with paragraph spacing
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card_data["body"].append({
                "type": "TextBlock",
                "text": paragraph,
                "wrap": True
            })
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating article template: {e}")
        return None

Let's create a basic article using this template:

In [ ]:
try:
    basic_article = create_article_template(
        title="Introducing Adaptive Cards for AI Agents",
        content="Adaptive Cards provide a powerful way for AI agents to communicate with users through rich, interactive content.\n\nWith our Agent Toolkit, creating these cards becomes even simpler, allowing your agents to focus on what matters: delivering valuable information to users in a format they can easily understand and interact with.",
        author="Adaptive Cards Team",
        date="February 27, 2023",
        actions=[
            {
                "type": "open_url",
                "title": "Learn More",
                "url": "https://adaptivecards.io/"
            }
        ]
    )
    
    display_card(basic_article, "Basic Article Card")
    
except Exception as e:
    print(f"Error creating basic article: {e}")

## Article with Image

Now let's add a header image to the article template:

In [ ]:
try:
    article_with_image = create_article_template(
        title="The Future of AI Communication",
        content="AI communication is rapidly evolving beyond simple text responses. Modern AI systems can now present information in rich, structured formats that improve user comprehension and engagement.\n\nAdaptive Cards represent a significant step forward, providing a consistent way for AI systems to generate rich, interactive content across platforms.",
        image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        author="Communications Team",
        date="February 26, 2023",
        actions=[
            {
                "type": "open_url",
                "title": "Learn More",
                "url": "https://adaptivecards.io/"
            },
            {
                "type": "submit",
                "title": "Subscribe",
                "data": {"action": "subscribe"}
            }
        ]
    )
    
    display_card(article_with_image, "Article with Image")
    
except Exception as e:
    print(f"Error creating article with image: {e}")

## Section-Based Content Organization

For longer content, organizing it into distinct sections improves readability. Let's create a template for section-based content:

In [ ]:
def create_sectioned_content_template(title, sections, image_url=None, author=None, date=None, actions=None):
    """Create a content template with organized sections.
    
    Args:
        title (str): Content title
        sections (list): List of section dictionaries with {title, content} keys
        image_url (str, optional): URL to a header image
        author (str, optional): Content author
        date (str, optional): Publication date
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create the card structure directly as JSON
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add header image if provided
        if image_url:
            card_data["body"].append({
                "type": "Image",
                "url": image_url,
                "altText": f"Image for {title}",
                "size": "stretch"
            })
        
        # Add main title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata if provided
        metadata = []
        if author:
            metadata.append(f"By {author}")
        if date:
            metadata.append(f"Published: {date}")
            
        if metadata:
            metadata_text = " · ".join(metadata)
            card_data["body"].append({
                "type": "TextBlock",
                "text": metadata_text,
                "isSubtle": True,
                "wrap": True
            })
        
        # Add each section
        for i, section in enumerate(sections):
            # Add divider between sections (except before the first section)
            if i > 0:
                card_data["body"].append({
                    "type": "Separator"
                })
                
            # Add section title
            card_data["body"].append({
                "type": "TextBlock",
                "text": section.get('title', f"Section {i+1}"),
                "size": "large",
                "weight": "bolder",
                "wrap": True
            })
            
            # Add section content with paragraph spacing
            content = section.get('content', '')
            paragraphs = content.split('\n\n')
            for paragraph in paragraphs:
                card_data["body"].append({
                    "type": "TextBlock",
                    "text": paragraph,
                    "wrap": True
                })
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        # Return a card object with to_json method
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating sectioned content template: {e}")
        return None

Let's create a sectioned article using this template:

In [ ]:
try:
    sectioned_article = create_sectioned_content_template(
        title="Guide to Adaptive Cards for AI Communication",
        sections=[
            {
                "title": "What are Adaptive Cards?",
                "content": "Adaptive Cards are a platform-agnostic way to create rich, interactive content. They allow you to design a card once and deploy it across multiple platforms with automatic styling to fit each platform's design language."
            },
            {
                "title": "Benefits for AI Agents",
                "content": "For AI agents, Adaptive Cards provide a structured way to present information that is more engaging than plain text.\n\nThey allow for the inclusion of images, formatted text, and interactive elements that can collect user input."
            },
            {
                "title": "Getting Started",
                "content": "To start using Adaptive Cards with your AI agents, you'll need to install the Adaptive Cards Toolkit and familiarize yourself with the basic components.\n\nThe toolkit makes it easy to create cards programmatically or through templates."
            }
        ],
        image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        author="Documentation Team",
        date="February 25, 2023",
        actions=[
            {
                "type": "open_url",
                "title": "View Documentation",
                "url": "https://adaptivecards.io/documentation/"
            },
            {
                "type": "submit",
                "title": "Try It Now",
                "data": {"action": "try_demo"}
            }
        ]
    )
    
    # Display the card
    display_card(sectioned_article, "Sectioned Article Card")
    
except Exception as e:
    print(f"Error creating sectioned article: {e}")

## Enhanced Content Layout

We can create a more advanced layout that combines various elements in a visually appealing way:

In [ ]:
def create_enhanced_article_template(title, content, features=None, image_url=None, author=None, date=None, actions=None):
    """Create an enhanced article template with features highlighting.
    
    Args:
        title (str): Article title
        content (str): Main article content
        features (list, optional): List of feature dictionaries with {title, description, icon_url} keys
        image_url (str, optional): URL to a header image
        author (str, optional): Article author
        date (str, optional): Publication date
        actions (list, optional): List of action dictionaries
        
    Returns:
        Object: Card object with to_json method
    """
    try:
        # Create the card structure directly as JSON
        card_data = {
            "type": "AdaptiveCard",
            "version": "1.5",
            "body": []
        }
        
        # Add header image if provided
        if image_url:
            card_data["body"].append({
                "type": "Image",
                "url": image_url,
                "altText": f"Image for {title}",
                "size": "stretch"
            })
        
        # Add title
        card_data["body"].append({
            "type": "TextBlock",
            "text": title,
            "size": "extraLarge",
            "weight": "bolder",
            "wrap": True
        })
        
        # Add metadata if provided
        metadata = []
        if author:
            metadata.append(f"By {author}")
        if date:
            metadata.append(f"Published: {date}")
            
        if metadata:
            metadata_text = " · ".join(metadata)
            card_data["body"].append({
                "type": "TextBlock",
                "text": metadata_text,
                "isSubtle": True,
                "wrap": True
            })
            
        # Add main content
        paragraphs = content.split('\n\n')
        for paragraph in paragraphs:
            card_data["body"].append({
                "type": "TextBlock",
                "text": paragraph,
                "wrap": True
            })
        
        # Add features section if provided
        if features and len(features) > 0:
            # Add a separator
            card_data["body"].append({
                "type": "Separator"
            })
            
            # Add features heading
            card_data["body"].append({
                "type": "TextBlock",
                "text": "Key Features",
                "size": "large",
                "weight": "bolder",
                "wrap": True
            })
            
            # Add features as a list of containers
            for feature in features:
                # Create container for each feature
                feature_container = {
                    "type": "Container",
                    "style": "emphasis",
                    "items": []
                }
                
                # If we have an icon, create a two-column layout
                if feature.get('icon_url'):
                    feature_container["items"].append({
                        "type": "ColumnSet",
                        "columns": [
                            {
                                "type": "Column",
                                "width": "auto",
                                "items": [
                                    {
                                        "type": "Image",
                                        "url": feature.get('icon_url'),
                                        "altText": f"Icon for {feature.get('title', 'feature')}",
                                        "size": "small"
                                    }
                                ]
                            },
                            {
                                "type": "Column",
                                "width": "stretch",
                                "items": [
                                    {
                                        "type": "TextBlock",
                                        "text": feature.get('title', 'Feature'),
                                        "weight": "bolder",
                                        "wrap": True
                                    },
                                    {
                                        "type": "TextBlock",
                                        "text": feature.get('description', ''),
                                        "wrap": True
                                    }
                                ]
                            }
                        ]
                    })
                else:
                    # Without icon, use a simpler layout
                    feature_container["items"].append({
                        "type": "TextBlock",
                        "text": feature.get('title', 'Feature'),
                        "weight": "bolder",
                        "wrap": True
                    })
                    feature_container["items"].append({
                        "type": "TextBlock",
                        "text": feature.get('description', ''),
                        "wrap": True
                    })
                
                # Add the feature container to the card
                card_data["body"].append(feature_container)
        
        # Add actions if provided
        if actions:
            card_data["actions"] = []
            for action in actions:
                if action.get('type') == 'open_url':
                    card_data["actions"].append({
                        "type": "Action.OpenUrl",
                        "title": action.get('title', 'Open'),
                        "url": action.get('url', '#')
                    })
                elif action.get('type') == 'submit':
                    card_data["actions"].append({
                        "type": "Action.Submit",
                        "title": action.get('title', 'Submit'),
                        "data": action.get('data', {})
                    })
        
        # Create a custom card object with to_json method
        class CardObject:
            def __init__(self, card_data):
                self.card_data = card_data
                
            def to_json(self):
                return json.dumps(self.card_data)
        
        return CardObject(card_data)
    
    except Exception as e:
        print(f"Error creating enhanced article template: {e}")
        return None

Let's create an enhanced article with feature highlights:

In [ ]:
try:
    enhanced_article = create_enhanced_article_template(
        title="Introducing the New Adaptive Cards Toolkit",
        content="We're excited to announce the release of our new Adaptive Cards Toolkit for AI agents. This toolkit provides a comprehensive set of tools for creating rich, interactive content that works across platforms.\n\nThe toolkit is designed to be easy to use, with a focus on creating cards programmatically or through templates. It includes components for creating common card types, validating cards before delivery, and sending cards to various platforms.",
        features=[
            {
                "title": "Cross-Platform Compatibility",
                "description": "Create cards once and deploy them across Teams, Outlook, and other platforms.",
                "icon_url": "https://adaptivecards.io/content/send.png"
            },
            {
                "title": "Rich Content Support",
                "description": "Add images, formatted text, and interactive elements to your cards.",
                "icon_url": "https://adaptivecards.io/content/person.png"
            },
            {
                "title": "Easy Integration",
                "description": "Simple API designed specifically for AI agents and automation scenarios.",
                "icon_url": "https://adaptivecards.io/content/checklist.png"
            }
        ],
        image_url="https://adaptivecards.io/content/adaptive-card-50.png",
        author="Product Team",
        date="February 24, 2023",
        actions=[
            {
                "type": "open_url",
                "title": "Get Started",
                "url": "https://adaptivecards.io/"
            },
            {
                "type": "submit",
                "title": "Request Demo",
                "data": {"action": "request_demo"}
            }
        ]
    )
    
    # Display the card
    display_card(enhanced_article, "Enhanced Article with Features")
    
except Exception as e:
    print(f"Error creating enhanced article: {e}")

## Next Steps

In this notebook, we've covered content templates for presenting information in a structured, readable format. We've learned how to create basic article templates, organize content into sections, and create enhanced layouts with feature highlights.

In the next notebook, 05c_toolkit_rich_media.ipynb, we'll explore more advanced content templates that incorporate additional media elements, rich metadata, and dynamic content rendering systems.